In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [4]:
pip install llama-cpp-python 

  Using cached llama_cpp_python-0.3.9.tar.gz (67.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Failed to build llama-cpp-python
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.11.3 using CMake 4.0.2 (wheel)
      *** Configuring CMake...
      2025-05-31 09:44:51,749 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\oarti\AppData\Local\Temp\tmp05zai5eu\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error or

In [5]:
import pandas as pd
df = pd.read_csv('../data/top_rated_wines.csv')
df = df[df['variety'].notna()] # remove os valores NaN
data = df.to_dict('records')
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


In [6]:
# A gente importa o QdrantClient pra conseguir conversar com o Qdrant, que é tipo um banco de dados
# especial pra guardar e buscar coisas por similaridade (tipo, "ache coisas parecidas com essa").
from qdrant_client import QdrantClient

# Importamos 'models' porque ele tem umas "receitas" e "definições" que o Qdrant precisa
# pra saber como organizar e entender as informações que a gente vai mandar pra ele.
from qdrant_client import models

# Trazemos o SentenceTransformer pra ele nos ajudar a transformar textos (frases, palavras)
# em números que o computador entende. Assim, o Qdrant consegue comparar um texto com outro.
from sentence_transformers import SentenceTransformer

In [7]:
"""
Instancia um objeto SentenceTransformer, 
# carregando o modelo 'all-MiniLM-L6-v2'. 
Este modelo é otimizado para gerar embeddings de sentenças,
convertendo texto em vetores numéricos de alta dimensionalidade
que capturam o significado semântico, sendo eficiente em termos
de desempenho e tamanho.
"""

encoder = SentenceTransformer('all-MiniLM-L6-v2') 

In [8]:
qdrant = QdrantClient(":memory:") 
# Inicia um banco de dados temporário na memória RAM .

In [9]:
qdrant.recreate_collection(
    collection_name="top_wines", 
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), 
        # Pega o tamanho exato dos vetores que o 'encoder' vai gerar
        distance=models.Distance.COSINE 
        # Define como a similaridade entre os vetores será calculada 
        # (nesse caso, por similaridade de cosseno)
    )
)

C:\Users\oarti\AppData\Local\Temp\ipykernel_2808\2759648202.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [10]:
# Mágica da vetorização!!!

qdrant.upload_points(
    collection_name="top_wines", # Armazena os dados na coleção "top_wines".
    points=[
        models.PointStruct(
            id=idx, # Dá um número de identificação único para cada vinho.
            vector=encoder.encode(doc["notes"]).tolist(), # Pega a descrição do vinho e a transforma em um "código numérico" (vetor).
            payload=doc, # Guarda todas as informações originais do vinho junto com o código.
        ) for idx, doc in enumerate(data) # Faz isso para cada vinho que temos na lista 'data'.
    ])

In [11]:
user_prompt = "Suggest me a Malbec wine from Argentina"

# Busca por vinhos com processamento de linguagem natural

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    # Transforma a pergunta do usuário em um vetor numérico
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

C:\Users\oarti\AppData\Local\Temp\ipykernel_2808\3773665370.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [12]:
search_results = [hit.payload for hit in hits] 
# Cria uma lista nova com as informações dos vinhos encontrados.

In [13]:
from openai import OpenAI
# Importa a classe OpenAI para interface com APIs de modelos de linguagem,
# incluindo servidores locais compatíveis.

client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # Define o endpoint da API para o LLM local.
    api_key="sk-no-key-required"         # Utiliza uma chave de API genérica, comum para setups locais.
)

completion = client.chat.completions.create(
    model="llama-3.2-3B-Instruct.Q6_K", 
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        # Configura o comportamento do LLM, definindo-o como um especialista em vinhos.
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        # Inclui a query original do usuário no histórico da conversa.
        {"role": "assistant", "content": str(search_results)}
        # Injeta os resultados da busca por similaridade como contexto adicional para o LLM.
    ]
)

print(completion.choices[0].message) # Exibe a resposta gerada pelo LLM.

ChatCompletionMessage(content="What a great selection! Based on your request for an amazing Malbec wine from Argentina, I highly recommend the **Catena Zapata Adrianna Vineyard Malbec 2004**. This wine is a standout among the three options, with a perfect balance of elegance, complexity, and power.\n\nThe Adrianna Vineyard Malbec is a single-vineyard wine from the Gualtallary district, known for its unique terroir and exceptional quality. The wine is aged for 18 months in new French oak, which adds a rich, velvety texture and subtle oak notes to the wine.\n\nThe flavor profile is characterized by dark fruit flavors, such as black cherry and blackberry, with hints of wood smoke, game, and spice. The tannins are silky and refined, making the wine a pleasure to drink.\n\nThis wine is a great representation of the best that Argentine Malbec has to offer, with a long aging potential of 6-8 years. It's a wine that will evolve and improve with time, but is also enjoyable to drink now.\n\nIf y